In [1]:
import os, sys
import logging
import glob
import json
import itertools
import collections
import importlib
import tempfile
from typing import *

import tqdm

import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats

import anndata as ad
import scanpy as sc

import torch
import torch.nn as nn
import skorch
import skorch.helper

from transformers import BertModel, BertForMaskedLM, BertTokenizer, FeatureExtractionPipeline

SRC_DIR = os.path.join(os.path.dirname(os.getcwd()), "tcr")
assert os.path.isdir(SRC_DIR), f"Cannot find src dir: {SRC_DIR}"
sys.path.append(SRC_DIR)
import custom_metrics
import data_loader as dl
import featurization as ft
import canonical_models as models
import model_utils
import plot_utils
import utils
MODEL_DIR = os.path.join(SRC_DIR, "models")
sys.path.append(MODEL_DIR)
import transformer_custom as trans
import conv

DEVICE = utils.get_device(3)
TRAINED_MODEL_DIR = "/home/wukevin/projects/tcr/tcr_models"
SCRIPTS_DIR = os.path.join(
    os.path.dirname(SRC_DIR),
    "scripts",
)
assert os.path.isdir(SCRIPTS_DIR)

PLOT_DIR = os.path.join(os.path.dirname(SRC_DIR), "plots/pird_antigen_cv")
assert os.path.isdir(PLOT_DIR)

In [2]:
# Load in the PIRD dataset
pird_data = dl.load_pird(with_antigen_only=True)
pird_data = pird_data.loc[~pd.isnull(pird_data['CDR3.beta.aa'])]
pird_data

INFO:root:PIRD data 0.1655 data labelled with antigen sequence
INFO:root:PIRD: Removing 95 entires with non amino acid residues
INFO:root:Entries with antigen sequence: 8429/51044
INFO:root:Unique antigen sequences: 73
INFO:root:PIRD data TRA/TRB instances: Counter({'TRB': 46428, 'TRA': 4011, 'TRA-TRB': 605})
INFO:root:PIRD entries with TRB sequence: 4607
INFO:root:PIRD entries with TRB sequence: 47040
INFO:root:PIRD entries with TRA and TRB:  605


,ICDname,Disease.name,Category,Antigen,Antigen.sequence,HLA,Locus,CDR3.alpha.aa,CDR3.beta.aa,CDR3.alpha.nt,...,Cell.subtype,Prepare.method,Evaluate.method,Case.num,Control.type,Control.num,Filteration,Journal,Pubmed.id,Grade
0,A15,Tuberculosis,Pathogen,CFP10,TAAQAAVVRFQEAAN,DRB1*15:03,TRA-TRB,CIEHTNSGGSNYKLTF,CASSLEETQYF,NaN,...,CD4,Multiple PCR,Antigen-specific ex vivo proliferation,22.0,NaN,NaN,NaN,Nature,28636589,5
1,A15,Tuberculosis,Pathogen,CFP10,TAAQAAVVRFQEAAN,DRB1*15:03,TRA-TRB,CIVHTNSGGSNYKLTF,CASSPEETQYF,NaN,...,CD4,Multiple PCR,Antigen-specific ex vivo proliferation,22.0,NaN,NaN,NaN,Nature,28636589,5
2,A15,Tuberculosis,Pathogen,CFP10,TAAQAAVVRFQEAAN,DRB1*15:03,TRA-TRB,CIVKTNSGGSNYKLTF,CASSFEETQYF,NaN,...,CD4,Multiple PCR,Antigen-specific ex vivo proliferation,22.0,NaN,NaN,NaN,Nature,28636589,5
1845,A15,Tuberculosis,Pathogen,Rv1195,ADTLQSIGATTVASN,DRB1*15:03,TRA-TRB,CAGAGGGGFKTIF,CASSVALASGANVLTF,NaN,...,CD4,Multiple PCR,Antigen-specific ex vivo proliferation,22.0,NaN,NaN,NaN,Nature,28636589,5
1846,A15,Tuberculosis,Pathogen,Rv1195,ADTLQSIGATTVASN,DRB1*15:03,TRA-TRB,CAGPTGGSYIPTF,CASSVALATGEQYF,NaN,...,CD4,Multiple PCR,Antigen-specific ex vivo proliferation,22.0,NaN,NaN,NaN,Nature,28636589,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51124,Z88,Allergy,Allergy,Beryllium sulfate,FWIDLFETIG,DPB1*02:01,TRB,NaN,CASSLSQGGDTQYF,NaN,...,CD4,NaN,Selection of antigen- specific T cells using p...,NaN,NaN,NaN,NaN,J Immunol,24719461,4
51125,Z88,Allergy,Allergy,Beryllium sulfate,FWIDLFETIG,DPB1*02:01,TRB,NaN,CASSLSQGGEKLF,NaN,...,CD4,NaN,Selection of antigen- specific T cells using p...,NaN,NaN,NaN,NaN,J Immunol,24719461,4
51126,Z88,Allergy,Allergy,Beryllium sulfate,FWIDLFETIG,DPB1*02:01,TRB,NaN,CASSLSQGGRPMF,NaN,...,CD4,NaN,Selection of antigen- specific T cells using p...,NaN,NaN,NaN,NaN,J Immunol,24719461,4
51127,Z88,Allergy,Allergy,Beryllium sulfate,FWIDLFETIG,DPB1*02:01,TRB,NaN,CASSMGQGGETQYF,NaN,...,CD4,NaN,Selection of antigen- specific T cells using p...,NaN,NaN,NaN,NaN,J Immunol,24719461,4


In [1]:
REDUCE_LR_ON_PLATEAU_PARAMS = {
    "mode": "min",
    "factor": 0.1,
    "patience": 10,
    "min_lr": 1e-6,
}

def convnet_on_antigen(antigen:str) -> Tuple[str, float]:
    """
    Train a convnet on the antigen and return Tuple[antigen, test_auprc]
    If invalid, return nan
    """
    pird_pos_table = pird_data.loc[pird_data['Antigen.sequence'] == antigen]
    #
    pird_pos_trbs = list(pird_pos_table['CDR3.beta.aa'])
    if len(pird_pos_trbs) < 20:
        return antigen, np.nan
    neg_trbs = dl.sample_unlabelled_tcrdb_trb(len(pird_pos_table) * 5)
    
    full_dset = dl.TCRSupervisedIdxDataset(
        pird_pos_trbs + neg_trbs,
        np.array([1] * len(pird_pos_trbs) + [0] * len(neg_trbs))
    )
    train_dset = dl.DatasetSplit(full_dset, split='train', valid=0.0, test=0.3)
    test_dset = dl.DatasetSplit(full_dset, split='test', valid=0.0, test=0.3)
    
    torch.manual_seed(42)
    with tempfile.TemporaryDirectory() as tmpdir:
        logging.info(f"Running in temporary dir: {tmpdir}")
        net = skorch.NeuralNet(
            module=conv.OnePartConvNet,
            module__use_embedding=True,
            module__n_output=2,
            module__max_input_len=full_dset.max_len,
            criterion=torch.nn.CrossEntropyLoss,
            optimizer=torch.optim.Adam,
            max_epochs=250,
            batch_size=512,
            lr=1e-3,
            callbacks=[
                skorch.callbacks.LRScheduler(
                    policy=torch.optim.lr_scheduler.ReduceLROnPlateau,
                    mode='min',
                    factor=0.1,
                    patience=10,
                    min_lr=1e-6,
                ),
                skorch.callbacks.GradientNormClipping(gradient_clip_value=5),
                skorch.callbacks.EpochScoring(
                    "average_precision",
                    lower_is_better=False,
                    on_train=False,
                    name="valid_auprc",
                ),
                skorch.callbacks.EarlyStopping(
                    patience=25,
                    monitor="valid_auprc",
                    lower_is_better=False,
                ),
                skorch.callbacks.Checkpoint(  # Seems to cause errors if placed before scoring
                    dirname=tmpdir,
                    fn_prefix="net_",
                    monitor="valid_auprc_best",
                ),
            ],
            device=3,
        )
        net.classes_ = np.unique(test_dset.all_labels())
        net.fit(train_dset)
        # Restore best weights and evaluate on test set
        cp = skorch.callbacks.Checkpoint(dirname=tmpdir, fn_prefix="net_")
#         net.load_params(checkpoint=cp)
        test_truth = test_dset.all_labels()
        test_preds = net.predict_proba(test_dset)[:, 1]
        test_auprc = metrics.average_precision_score(test_truth, test_preds)
    return antigen, test_auprc
    
convnet_on_antigen("FPRPWLHGL")

NameError: name 'Tuple' is not defined

In [4]:
antigen_auprc_pairs = [convnet_on_antigen(a) for a in utils.dedup(pird_data['Antigen.sequence'])]
len(antigen_auprc_pairs)

INFO:root:Got maximum length of 22
INFO:root:Split train with 970 examples
INFO:root:Split test with 416 examples
INFO:root:Running in temporary dir: /tmp/tmphsl1uj0n


  epoch    train_loss    valid_auprc    valid_loss    cp     dur
-------  ------------  -------------  ------------  ----  ------
      1        0.8163         0.1578        0.6622     +  0.0255
      2        0.5344         0.2012        0.6552     +  0.0243
      3        0.4366         0.3027        0.6484     +  0.0392
      4        0.3731         0.3800        0.6336     +  0.0337
      5        0.3353         0.4502        0.6101     +  0.0375
      6        0.3127         0.4901        0.5762     +  0.0313
      7        0.2933         0.5338        0.5341     +  0.0237
      8        0.2754         0.5463        0.4881     +  0.0239
      9        0.2593         0.5553        0.4436     +  0.0241
     10        0.2444         0.5848        0.4058     +  0.0240
     11        0.2302         0.6045        0.3769     +  0.0237
     12        0.2169         0.6097        0.3565     +  0.0235
     13        0.2047         0.6114        0.3422     +  0.0237
     14        0.1936    

INFO:root:Got maximum length of 20
INFO:root:Split train with 239 examples
INFO:root:Split test with 103 examples
INFO:root:Running in temporary dir: /tmp/tmp2xkvv5bx


  epoch    train_loss    valid_auprc    valid_loss    cp     dur
-------  ------------  -------------  ------------  ----  ------
      1        1.1928         0.1823        0.7052     +  0.0096
      2        0.8366         0.1765        0.6801        0.0095
      3        0.5950         0.1901        0.6565     +  0.0092
      4        0.4429         0.2986        0.6340     +  0.0092
      5        0.3504         0.4274        0.6111     +  0.0091
      6        0.2899         0.4686        0.5882     +  0.0092
      7        0.2477         0.5115        0.5653     +  0.0091
      8        0.2154         0.5510        0.5419     +  0.0092
      9        0.1892         0.5595        0.5181     +  0.0091
     10        0.1666         0.6419        0.4944     +  0.0090
     11        0.1467         0.6849        0.4707     +  0.0092
     12        0.1290         0.7359        0.4477     +  0.0091
     13        0.1138         0.8102        0.4255     +  0.0092
     14        0.1009    

INFO:root:Got maximum length of 20
INFO:root:Split train with 239 examples
INFO:root:Split test with 103 examples
INFO:root:Running in temporary dir: /tmp/tmp8gzwymp9


...

  epoch    train_loss    valid_auprc    valid_loss    cp     dur
-------  ------------  -------------  ------------  ----  ------
      1        1.1607         0.1111        0.7152     +  0.0076
      2        0.7629         0.2000        0.6928     +  0.0077
      3        0.4940         0.2500        0.6723     +  0.0072
      4        0.3268         0.5000        0.6515     +  0.0071
      5        0.2277         1.0000        0.6314     +  0.0071
      6        0.1682         1.0000        0.6102        0.0072
      7        0.1298         1.0000        0.5866        0.0072
      8        0.1025         1.0000        0.5615        0.0115
      9        0.0820         1.0000        0.5352        0.0125
     10        0.0663         1.0000        0.5078        0.0100
     11        0.0543         1.0000        0.4806        0.0087
     12        0.0451         1.0000        0.4541        0.0079
     13        0.0381         1.0000        0.4285        0.0075
     14        0.0329    

INFO:root:Got maximum length of 20
INFO:root:Split train with 945 examples
INFO:root:Split test with 405 examples
INFO:root:Running in temporary dir: /tmp/tmp3oqrre77


  epoch    train_loss    valid_auprc    valid_loss    cp     dur
-------  ------------  -------------  ------------  ----  ------
      1        1.0622         0.1648        0.6890     +  0.0355
      2        0.5717         0.5677        0.6375     +  0.0373
      3        0.3811         0.6276        0.5844     +  0.0248
      4        0.3111         0.6655        0.5287     +  0.0249
      5        0.2792         0.6870        0.4744     +  0.0258
      6        0.2589         0.7014        0.4260     +  0.0260
      7        0.2417         0.7059        0.3854     +  0.0258
      8        0.2261         0.7115        0.3529     +  0.0259
      9        0.2116         0.7151        0.3279     +  0.0260
     10        0.1983         0.7175        0.3094     +  0.0259
     11        0.1865         0.7199        0.2958     +  0.0249
     12        0.1759         0.7258        0.2854     +  0.0232
     13        0.1667         0.7289        0.2771     +  0.0230
     14        0.1588    

     99        0.0054         0.7642        0.2580        0.0237
    100        0.0052         0.7640        0.2584        0.0236
Stopping since valid_auprc has not improved in the last 25 epochs.


73

In [5]:
antigen_auprc_pairs = [pair for pair in antigen_auprc_pairs if not np.isnan(pair[1])]
len(antigen_auprc_pairs)

26

In [9]:
df = pd.DataFrame(
    [pair[1] for pair in antigen_auprc_pairs],
    index=[pair[0] for pair in antigen_auprc_pairs],
    columns=['ConvNet']
)
df

,ConvNet
LLWNGPMAV,0.714525
RPRGEVRFL,0.804326
ATDALMTGY,0.968797
HSKKKCDEL,0.777029
KAFSPEVIPMF,0.699305
KRWIILGLNK,0.434463
TPQDLNTML,0.667731
EIYKRWII,0.341290
HPKVSSEVHI,0.613221
IIKDYGKQM,0.716234


In [10]:
df.to_csv("antigen_cv_convnet_baseline.csv")